In [14]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai

In [1]:
import os
# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
# os.environ["TAVILY_API_KEY"] = "your-api-key"
# os.environ["OPENAI_API_KEY"] = 'your-api-key'

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

search = TavilySearchResults()

search.invoke("what is the weather in SF")

[{'title': 'Weather in San Francisco',
  'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744902463, 'localtime': '2025-04-17 08:07'}, 'current': {'last_updated_epoch': 1744902000, 'last_updated': '2025-04-17 08:00', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 174, 'wind_dir': 'S', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 100, 'feelslike_c': 11.0, 'feelslike_f': 51.7, 'windchill_c': 10.4, 'windchill_f': 50.7, 'heatindex_c': 11.0, 'heatindex_f': 51.7, 'dewpoint_c': 9.7, 'dewpoint_f': 49.4, 'vis_km': 14.0, 'vis_miles': 8.0, 'uv': 0.2, 'gust_mph': 7.1, 'gust_kph': 11.4}}",
  'score': 

In [2]:
from langchain_core.tools import tool

@tool 
def create_file(content: str, filename: str):
    """
    Create a file with the given content.
    """
    with open(filename, "w") as f:
        f.write(content)


In [3]:
create_file.invoke({"content": "file content", "filename": "file.txt"}) 

In [4]:
tools = [search, create_file]

In [5]:
# Set up the model
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [6]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [8]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [9]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Write a file named pancakes.txt with updated information from the web about why pancakes are the best."})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'why pancakes are the best'}`


[{'title': "Writers' Block: No Need to Waffle, It's Clear that Pancakes are the ...", 'url': 'https://www.cunesower.com/writers-block-no-need-to-waffle-its-clear-that-pancakes-are-the-better-choice/', 'content': 'Writers’ Block: No Need to Waffle, It’s Clear that Pancakes are the Better Choice\n\nMeredith Boster\nSower Staff\n\n\n\nPancakes are one of the best foods, if not the best food ever to exist.\n\nThey are easy to make, easy to stack and easy to eat. They have an infinite number of combinations that go with them, they’re filling, and they can be eaten for breakfast, lunch or dinner. [...] Pancakes are time efficient, delicious, filling and versatile. THAT is why they are superior to any other food.\n\nSimilar articles\n\nStudents find enjoyment in majors across CUNE’s 11 departments\n\nCartoon by Nora Betts: “Sanctifying Weather”\n\nPlease leave a repl

{'input': 'Write a file named pancakes.txt with updated information from the web about why pancakes are the best.',
 'output': 'I have created a file named `pancakes.txt` with updated information about why pancakes are considered the best.'}